In [1]:
from dask.distributed import Client, as_completed
import numpy as np
import time
import os
from pyretis.setup import create_simulation
from pyretis.core.tis import shoot
from pyretis.core.retis import retis_swap_zero # need to disable "add_path_data()"
from pyretis.inout.settings import parse_settings_file
from infretis import REPEX_state, calc_cv_vector

In [2]:
# Connect to scheduler
from dask.distributed import Client
n_workers = 2
client = Client(n_workers=n_workers)
client

2022-10-17 13:59:39,757 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-9hwh1zpc', purging
2022-10-17 13:59:39,757 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-oxnrj4y_', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 48,Total memory: 31.35 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38131,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 48
Started: Just now,Total memory: 31.35 GiB
Comm: tcp://127.0.0.1:33525,Total threads: 24
Dashboard: http://127.0.0.1:37201/status,Memory: 15.67 GiB
Nanny: tcp://127.0.0.1:33573,


In [7]:
# Set up simulation variables
INP = './retis.rst'
sim_settings = parse_settings_file(INP)
sim = create_simulation(sim_settings)
sim.set_up_output(sim_settings)
sim.initiate(sim_settings)


CREATING  ENSEMBLE  0
Initializing unit system.
Creating system from settings.
Creating force field.
Created units: "gromacs".
Created engine "GROMACS engine" from settings.

CREATING  ENSEMBLE  1
Initializing unit system.
Creating system from settings.
Creating force field.
Created units: "gromacs".
Created engine "GROMACS engine" from settings.

CREATING  ENSEMBLE  2
Initializing unit system.
Creating system from settings.
Creating force field.
Created units: "gromacs".
Created engine "GROMACS engine" from settings.

CREATING  ENSEMBLE  3
Initializing unit system.
Creating system from settings.
Creating force field.
Created units: "gromacs".
Created engine "GROMACS engine" from settings.

CREATING  ENSEMBLE  4
Initializing unit system.
Creating system from settings.
Creating force field.
Created units: "gromacs".


CRITICAL:pyretis.inout.fileio:Could not open file "load/000/energy.txt" for reading
CRITICAL:pyretis.inout.fileio:I/O error ({2}): {No such file or directory}
CRITICAL:pyretis.initiation.initiate_load:Initial path for [0^-] ends at the wrong interface!


Created engine "GROMACS engine" from settings.
Will initiate paths using method "load".

Loading data for path path ensemble [0^-]
Copying trajectory files.
Loading order parameters.
Creating path from files.
Loading energy data.
Could not read file: load/000/energy.txt
Energies are not set.
Sorting input files
Found initial path for [0^-]:
- Path with length 179 (max: None)
- Order parameter max: (0.270052, 0)
- Order parameter min: (0.263395, 89)
- Start 0.270052
- End 0.270052
- Status: The path has been accepted
- Generated: Path was loaded from unformatted external file(s)
- Weight: 1.0



CRITICAL:pyretis.inout.fileio:Could not open file "load/001/energy.txt" for reading
CRITICAL:pyretis.inout.fileio:I/O error ({2}): {No such file or directory}
CRITICAL:pyretis.initiation.initiate_load:Initial path for [0^+] starts at the wrong interface!


# Results for path ensemble [0^-] at cycle 0:
# Generated path with status "ACC", move "unformatted ext file load" and length 179.
# Order parameter max was: 0.270052 at index 0.
# Order parameter min was: 0.263395 at index 89.
# Path ensemble statistics:
# Ensemble     Cycle    Accepted         BWI         BTL         FTL         BTX         FTX
     [0^-]         0           1           0           0           0           0           0


Loading data for path path ensemble [0^+]
Copying trajectory files.
Loading order parameters.
Creating path from files.
Loading energy data.
Could not read file: load/001/energy.txt
Energies are not set.
Sorting input files
Found initial path for [0^+]:
- Path with length 109 (max: None)
- Order parameter max: (0.279931, 54)
- Order parameter min: (0.26981, 0)
- Start 0.26981
- End 0.26981
- Status: The path has been accepted
- Generated: Path was loaded from unformatted external file(s)
- Weight: 1.0



...

CRITICAL:pyretis.inout.fileio:Could not open file "load/003/energy.txt" for reading
CRITICAL:pyretis.inout.fileio:I/O error ({2}): {No such file or directory}
CRITICAL:pyretis.initiation.initiate_load:Initial path for [2^+] ends at the wrong interface!


# Results for path ensemble [1^+] at cycle 0:
# Generated path with status "ACC", move "unformatted ext file load" and length 291.
# Order parameter max was: 0.314728 at index 145.
# Order parameter min was: 0.269903 at index 0.
# Path ensemble statistics:
# Ensemble     Cycle    Accepted         BWI         BTL         FTL         BTX         FTX
     [1^+]         0           1           0           0           0           0           0


Loading data for path path ensemble [2^+]
Copying trajectory files.
Loading order parameters.
Creating path from files.
Loading energy data.
Could not read file: load/003/energy.txt
Energies are not set.
Sorting input files
Found initial path for [2^+]:
- Path with length 61 (max: None)
- Order parameter max: (0.307148, 30)
- Order parameter min: (0.269896, 0)
- Start 0.269896
- End 0.269896
- Status: The path has been accepted
- Generated: Path was loaded from unformatted external file(s)
- Weight: 1.0



CRITICAL:pyretis.inout.fileio:Could not open file "load/004/energy.txt" for reading
CRITICAL:pyretis.inout.fileio:I/O error ({2}): {No such file or directory}
CRITICAL:pyretis.initiation.initiate_load:Initial path for [3^+] ends at the wrong interface!


# Results for path ensemble [2^+] at cycle 0:
# Generated path with status "ACC", move "unformatted ext file load" and length 61.
# Order parameter max was: 0.307148 at index 30.
# Order parameter min was: 0.269896 at index 0.
# Path ensemble statistics:
# Ensemble     Cycle    Accepted         BWI         BTL         FTL         BTX         FTX
     [2^+]         0           1           0           0           0           0           0


Loading data for path path ensemble [3^+]
Copying trajectory files.
Loading order parameters.
Creating path from files.
Loading energy data.
Could not read file: load/004/energy.txt
Energies are not set.
Sorting input files
Found initial path for [3^+]:
- Path with length 101 (max: None)
- Order parameter max: (0.336438, 50)
- Order parameter min: (0.269994, 0)
- Start 0.269994
- End 0.269994
- Status: The path has been accepted
- Generated: Path was loaded from unformatted external file(s)
- Weight: 1.0

# Results for path ensemble [3^+] at cycle 0:


True

In [8]:
def run_md(ens_num, input_traj, settings, ensembles, pin, cycle):
    interfaces = settings['simulation']['interfaces']
    out = {'ensembles': [],                                                  
           'accepted_trajs': [],                                        
           'traj_vectors': [],
           'status': None,
           'pin': pin}
    path_numbers_old = []
    for traj0 in input_traj:
        path_numbers_old.append(traj0.path_number)
    out['path_numbers_old'] = path_numbers_old
    
    if len(ens_num) == 1:
        print('bling', ens_num)
        start_cond = ensembles[ens_num[0]+1]['path_ensemble'].start_condition
        tis_settings = settings['ensemble'][ens_num[0]+1]['tis'] 

        accept, trial, status = shoot(ensembles[ens_num[0]+1],
                                      tis_settings,
                                      start_cond)
        
        if accept:                                                                  
            out_traj = trial                                                        
        else:                                                                       
            out_traj = input_traj[0]
        if ens_num[0] < 0:
            interfaces = interfaces[0:1]
            print('cowboiiii', interfaces)
        cv_vector = calc_cv_vector(out_traj, interfaces)
                
        out['ensembles'] = list(ens_num)
        out['accepted_trajs'] = [out_traj]
        out['traj_vectors'] = [cv_vector]
        out['status'] = status
    else:
        print('blang')
        accept, trial, status = retis_swap_zero(ensembles, 
                                                settings,
                                                cycle)
        if accept:                                                                  
            out_traj = trial                                                        
        else:                                                                       
            out_traj = input_traj

        out['ensembles'] = list(ens_num)
        out['accepted_trajs'] = out_traj
        ifaces = [[interfaces[0:1]], interfaces]
        for traj, iface in zip(out_traj, ifaces):
            out['traj_vectors'].append(calc_cv_vector(traj, iface))     
        out['status'] = status

    return out


def print_path_info(state):
    ens_no = len(state._trajs[:-1])
    ens_str = [f'{i:03.0f}' for i in range(ens_no)]
    state_dic = {}
    ens_pwds = []
    
    for path_temp in state._trajs[:-1]:
        path_pwds = sorted(set([pp.particles.config[0] for pp in path_temp.phasepoints]))
        ens = path_pwds[0].split('/')[4]
        state_dic[ens] = {'pwds': [pwd.split('/')[-1] for pwd in path_pwds]}
        state_dic[ens]['path_number'] = path_temp.path_number

    for ens in ens_str:
        ens_pwds.append(sorted(os.listdir(f'./{ens}/accepted')))

    # check if state_paths correspond to path_pwds:
    for ens, string1 in zip(ens_str, ens_pwds):
        string0 = state_dic[ens]['pwds']
        if string0 != string1:
            print(string0, string1)
            print('warning! the state_paths does not correspond to the path_pwds!')
    
    # print current state
    print('===')
    print(f'ens\tp_num\tstate\t\t\tfiles')
    for idx, ens in enumerate(ens_str):
        print(f"{ens}\t{state_dic[ens]['path_number']}",
              f"\t{state.state[idx]}\t{state_dic[ens]['pwds']}")
    print('===')
    

In [9]:
#start some timing to keep track
walltime = 1*20 #(sec per hour)
interfaces = sim.settings['simulation']['interfaces']
size=len(interfaces)
start_time = time.time() 
current_time = start_time
traj_num_dic = {} # store pushed out trajs
traj_num_live = {} # store stored path locs
traj_num = 0
# Setup the infretis simulation
state = REPEX_state(size, minus=True)

for i in range(size-1):
    # we add all the i+ paths.
    path = sim.ensembles[i+1]['path_ensemble'].last_path
    path.path_number = traj_num
    state.add_traj(ens=i, traj=path,
                   valid=calc_cv_vector(path, interfaces),
                   count=False)
    traj_num_dic[traj_num] = {'weight': np.zeros(size+1), 
                              'adress':  set(kk.particles.config[0].split('salt')[-1] for kk in path.phasepoints)}
    traj_num_live[traj_num] = traj_num + 1
    traj_num += 1

# add minus path:
path = sim.ensembles[0]['path_ensemble'].last_path
path.path_number = traj_num
state.add_traj(ens=-1, traj=path, valid=(1,), count=False)
traj_num_dic[traj_num] = {'weight': np.zeros(size+1),
                          'adress':  set(kk.particles.config[0].split('salt')[-1] for kk in path.phasepoints)}
traj_num_live[traj_num] = 0 # the minus path is added to ens 0
traj_num += 1

# Define an iterator that is going to be our results queue
futures = as_completed(None, with_results=True)

# Start running
traj_i = 0

In [10]:
print('stored ensemble paths:')
print(' '.join([f'00{i}: {j},' for i, j in enumerate([sim0['path_ensemble'].last_path.path_number for sim0 in sim.ensembles])]))
print('saved ensemble paths:', [last_path0.path_number for last_path0 in state._trajs[:-1]])
print_path_info(state)
print(' ')

# submit the first batch of workers
for worker in range(n_workers):
    print(f'------- pipipi {traj_i} START -------')
    ens, input_traj = state.pick()
    print('Shooting in ensembles:', ' '.join([f'00{ens_num+1}' for ens_num in ens]),
          'with paths:', ' '.join([str(trajj.path_number) for trajj in input_traj]),
          'and worker:', worker)
    
    # assign gromacs resurces to workers:
    for ens_num, traj_inp in zip(ens, input_traj):
        ens_num += 1
        # pin = '0' if '0' in sim.ensembles[ens_num]['engine'].mdrun else '1'
        pin = str(worker)
        mdrun = sim.ensembles[ens_num]['engine'].mdrun.replace('0', f'{worker}')
        mdrun_c = sim.ensembles[ens_num]['engine'].mdrun_c.replace('0', f'{worker}')
        sim.ensembles[ens_num]['engine'].mdrun = mdrun
        sim.ensembles[ens_num]['engine'].mdrun_c = mdrun_c
        sim.ensembles[ens_num]['path_ensemble'].last_path = traj_inp.copy()
        
    # submit job
    fut = client.submit(run_md, ens, input_traj, sim.settings, sim.ensembles, pin, traj_i, pure=False)
    futures.add(fut)
    print(f'------- pipipi {traj_i} END -------')
    traj_i += 1
print()


# while loop
stopping = 100
while traj_i < stopping:
    
    # get output from finished worker
    output = next(futures)[1]
    ensembles = output['ensembles']
    out_trajs = output['accepted_trajs']
    traj_vectors = output['traj_vectors']
    status = output['status']
    path_numbers_old = output['path_numbers_old']
    pin = output['pin']
    
    print(f'------- pipipi {traj_i} START -------')
    
    # analyse and record worker data
    for ens_num, out_traj, traj_v, pn_old in zip(ensembles,
                                                 out_trajs,
                                                 traj_vectors,
                                                 path_numbers_old):
        
        # if path is new: number and save the path:
        if out_traj.path_number == None or status == 'ACC':
            out_traj.path_number = traj_num
            traj_num_dic[traj_num] = {'weight': np.zeros(size+1),
                                      'adress': set(kk.particles.config[0].split('salt')[-1] 
                                                    for kk in out_traj.phasepoints)}
            ens_save_idx = traj_num_live[pn_old]
            traj_num_live[traj_num] = ens_save_idx
            traj_num += 1
            
            sim.ensembles[ens_save_idx]['path_ensemble'].store_path(out_traj)
        state.add_traj(ens_num, out_traj, traj_v)
        
        # record weights 
        live_trajs = [traj.path_number for traj in state._trajs[:-1]] # state.live_paths()
        for live, last_prob in zip(live_trajs, state._last_prob[:-1]):
            traj_num_dic[live]['weight'] += last_prob
    
    # print information to screen
    print('shooted in ensembles:', ' '.join([f'00{ens_num+1}' for ens_num in ensembles]),
          'with paths:', ' '.join([str(pn_old) for pn_old in path_numbers_old]), '->', 
          ' '.join([str(trajj.path_number) for trajj in out_trajs]),
          'with status:', status, 'and worker:', pin)
    print_path_info(state)
    
    # chose ens and path for the next job
    ens, input_traj = state.pick()
    print('shooting in ensembles:', ' '.join([f'00{ens_num+1}' for ens_num in ens]),
          'with paths:', ' '.join([str(trajj.path_number) for trajj in input_traj]),
          'and worker:', pin)

    # assign gromacs resurces to workers:
    for ens_num, traj_inp in zip(ens, input_traj):
        ens_num += 1
        not_pin = '1' if pin == '0' else '0'
        mdrun = sim.ensembles[ens_num]['engine'].mdrun.replace(not_pin, pin)
        mdrun_c = sim.ensembles[ens_num]['engine'].mdrun_c.replace(not_pin, pin)
        sim.ensembles[ens_num]['engine'].mdrun = mdrun
        sim.ensembles[ens_num]['engine'].mdrun_c = mdrun_c
        sim.ensembles[ens_num]['path_ensemble'].last_path = traj_inp.copy()
    
    print(f'------- pipipi {traj_i} END -------')
        
    # cleanup check
    for ens_num in ens:
        sim.ensembles[ens_num+1]['engine'].clean_up()
    
    # submit job
    fut = client.submit(run_md, ens, input_traj, sim.settings, sim.ensembles, pin, traj_i, pure=False)
    futures.add(fut)
    traj_i += 1
    print()


stored ensemble paths:
000: 4, 001: 0, 002: 1, 003: 2, 004: 3,
saved ensemble paths: [4, 0, 1, 2, 3]
===
ens	p_num	state			files
000	4 	[1. 0. 0. 0. 0. 0.]	['zero.trr']
001	0 	[0. 1. 0. 0. 0. 0.]	['one.trr']
002	1 	[0. 1. 1. 1. 0. 0.]	['002_1_trajB.trr', '002_2_trajF.trr']
003	2 	[0. 1. 1. 1. 0. 0.]	['003_1_trajB.trr', '003_2_trajF.trr']
004	3 	[0. 1. 1. 1. 1. 0.]	['004_1_trajB.trr', '004_2_trajF.trr']
===
 
------- pipipi 0 START -------
Shooting in ensembles: 004 with paths: 3 and worker: 0
------- pipipi 0 END -------
------- pipipi 1 START -------
Shooting in ensembles: 003 with paths: 2 and worker: 1
------- pipipi 1 END -------

------- pipipi 2 START -------
shooted in ensembles: 003 with paths: 2 -> 5 with status: ACC and worker: 1
===
ens	p_num	state			files
000	4 	[1. 0. 0. 0. 0. 0.]	['zero.trr']
001	0 	[0. 1. 0. 0. 0. 0.]	['one.trr']
002	1 	[0. 1. 1. 1. 0. 0.]	['002_1_trajB.trr', '002_2_trajF.trr']
003	5 	[0. 1. 1. 1. 0. 0.]	['003_63_trajB.trr', '003_64_trajF.trr']
004	3 	[0

------- pipipi 16 START -------
shooted in ensembles: 000 001 with paths: 15 16 -> 17 18 with status: ACC and worker: 1
===
ens	p_num	state			files
000	17 	[1. 0. 0. 0. 0. 0.]	['000_68_trajB.trr', 'second.g96']
001	18 	[0. 1. 0. 0. 0. 0.]	['001_69_trajF.trr', 'second_last.g96']
002	8 	[0. 1. 1. 1. 0. 0.]	['002_65_trajB.trr', '002_66_trajF.trr']
003	5 	[0. 1. 1. 1. 0. 0.]	['003_63_trajB.trr', '003_64_trajF.trr']
004	6 	[0. 1. 1. 1. 1. 0.]	['004_54_trajB.trr', '004_55_trajF.trr']
===
shooting in ensembles: 000 001 with paths: 17 18 and worker: 1
------- pipipi 16 END -------

------- pipipi 17 START -------
shooted in ensembles: 002 with paths: 5 -> 5 with status: FTL and worker: 0
===
ens	p_num	state			files
000	17 	[1. 0. 0. 0. 0. 0.]	['000_68_trajB.trr', 'second.g96']
001	18 	[0. 1. 0. 0. 0. 0.]	['001_69_trajF.trr', 'second_last.g96']
002	8 	[0. 1. 1. 1. 0. 0.]	['002_65_trajB.trr', '002_66_trajF.trr']
003	5 	[0. 1. 1. 1. 0. 0.]	['003_63_trajB.trr', '003_64_trajF.trr']
004	6 	[0. 1. 1.

------- pipipi 31 START -------
shooted in ensembles: 003 with paths: 22 -> 22 with status: NCR and worker: 1
===
ens	p_num	state			files
000	29 	[1. 0. 0. 0. 0. 0.]	['000_88_trajB.trr', '000_89_trajF.trr']
001	30 	[0. 1. 1. 1. 0. 0.]	['002_82_trajB.trr', '002_83_trajF.trr']
002	22 	[0. 1. 1. 0. 0. 0.]	['002_72_trajB.trr', '002_73_trajF.trr']
003	25 	[0. 1. 1. 1. 0. 0.]	['002_82_trajB.trr', '002_83_trajF.trr']
004	21 	[0. 1. 1. 1. 1. 0.]	['004_78_trajB.trr', '004_79_trajF.trr']
===
shooting in ensembles: 002 with paths: 25 and worker: 1
------- pipipi 31 END -------

------- pipipi 32 START -------
shooted in ensembles: 004 with paths: 21 -> 31 with status: ACC and worker: 0
===
ens	p_num	state			files
000	29 	[1. 0. 0. 0. 0. 0.]	['000_88_trajB.trr', '000_89_trajF.trr']
001	30 	[0. 1. 1. 0. 0. 0.]	['002_82_trajB.trr', '002_83_trajF.trr']
002	22 	[0. 1. 1. 1. 0. 0.]	['002_72_trajB.trr', '002_73_trajF.trr']
003	25 	[0. 1. 1. 1. 0. 0.]	['002_82_trajB.trr', '002_83_trajF.trr']
004	31 	[0. 

------- pipipi 46 START -------
shooted in ensembles: 003 with paths: 40 -> 40 with status: NCR and worker: 1
===
ens	p_num	state			files
000	41 	[1. 0. 0. 0. 0. 0.]	['000_96_trajB.trr', '000_97_trajF.trr']
001	39 	[0. 1. 1. 0. 0. 0.]	['001_98_trajB.trr', '001_99_trajF.trr']
002	40 	[0. 1. 1. 0. 0. 0.]	['003_100_trajB.trr', '003_101_trajF.trr']
003	42 	[0. 1. 1. 1. 0. 0.]	['002_98_trajB.trr', '002_99_trajF.trr']
004	34 	[0. 1. 1. 1. 1. 0.]	['004_92_trajB.trr', '004_93_trajF.trr']
===
shooting in ensembles: 000 001 with paths: 41 39 and worker: 1
------- pipipi 46 END -------

------- pipipi 47 START -------
shooted in ensembles: 000 001 with paths: 41 39 -> 43 44 with status: ACC and worker: 1
===
ens	p_num	state			files
000	43 	[1. 0. 0. 0. 0. 0.]	['000_106_trajB.trr', 'second.g96']
001	44 	[0. 1. 1. 0. 0. 0.]	['001_107_trajF.trr', 'second_last.g96']
002	40 	[0. 1. 1. 0. 0. 0.]	['003_100_trajB.trr', '003_101_trajF.trr']
003	42 	[0. 1. 1. 1. 0. 0.]	['002_98_trajB.trr', '002_99_trajF.tr

------- pipipi 61 START -------
shooted in ensembles: 002 with paths: 42 -> 53 with status: ACC and worker: 1
===
ens	p_num	state			files
000	50 	[1. 0. 0. 0. 0. 0.]	['000_111_trajB.trr', 'second.g96']
001	51 	[0. 1. 1. 1. 0. 0.]	['001_112_trajF.trr', 'second_last.g96']
002	52 	[0. 1. 1. 0. 0. 0.]	['003_117_trajB.trr', '003_118_trajF.trr']
003	53 	[0. 1. 1. 1. 0. 0.]	['002_119_trajB.trr', '002_120_trajF.trr']
004	45 	[0. 1. 1. 1. 1. 0.]	['004_100_trajB.trr', '004_101_trajF.trr']
===
shooting in ensembles: 000 001 with paths: 50 52 and worker: 1
------- pipipi 61 END -------

------- pipipi 62 START -------
shooted in ensembles: 004 with paths: 45 -> 54 with status: ACC and worker: 0
===
ens	p_num	state			files
000	50 	[1. 0. 0. 0. 0. 0.]	['000_111_trajB.trr', 'second.g96']
001	51 	[0. 1. 1. 1. 0. 0.]	['001_112_trajF.trr', 'second_last.g96']
002	52 	[0. 1. 1. 0. 0. 0.]	['003_117_trajB.trr', '003_118_trajF.trr']
003	53 	[0. 1. 1. 1. 0. 0.]	['002_119_trajB.trr', '002_120_trajF.trr']
004	5

------- pipipi 76 START -------
shooted in ensembles: 001 with paths: 65 -> 67 with status: ACC and worker: 0
===
ens	p_num	state			files
000	64 	[1. 0. 0. 0. 0. 0.]	['000_125_trajB.trr', 'second.g96']
001	58 	[0. 1. 1. 1. 0. 0.]	['003_117_trajB.trr', '003_118_trajF.trr']
002	66 	[0. 1. 1. 0. 0. 0.]	['002_133_trajB.trr', '002_134_trajF.trr']
003	67 	[0. 1. 1. 1. 0. 0.]	['001_127_trajB.trr', '001_128_trajF.trr']
004	63 	[0. 1. 1. 1. 1. 0.]	['004_123_trajB.trr', '004_124_trajF.trr']
===
shooting in ensembles: 003 with paths: 58 and worker: 0
------- pipipi 76 END -------

------- pipipi 77 START -------
shooted in ensembles: 004 with paths: 63 -> 68 with status: ACC and worker: 1
===
ens	p_num	state			files
000	64 	[1. 0. 0. 0. 0. 0.]	['000_125_trajB.trr', 'second.g96']
001	58 	[0. 1. 1. 1. 0. 0.]	['003_117_trajB.trr', '003_118_trajF.trr']
002	66 	[0. 1. 1. 0. 0. 0.]	['002_133_trajB.trr', '002_134_trajF.trr']
003	67 	[0. 1. 1. 1. 0. 0.]	['001_127_trajB.trr', '001_128_trajF.trr']
004	68 	

------- pipipi 91 START -------
shooted in ensembles: 000 001 with paths: 76 69 -> 80 81 with status: ACC and worker: 1
===
ens	p_num	state			files
000	80 	[1. 0. 0. 0. 0. 0.]	['000_149_trajB.trr', 'second.g96']
001	81 	[0. 1. 1. 0. 0. 0.]	['001_150_trajF.trr', 'second_last.g96']
002	79 	[0. 1. 1. 0. 0. 0.]	['002_141_trajB.trr', '002_142_trajF.trr']
003	77 	[0. 1. 1. 1. 0. 0.]	['003_139_trajB.trr', '003_140_trajF.trr']
004	78 	[0. 1. 1. 1. 1. 0.]	['004_147_trajB.trr', '004_148_trajF.trr']
===
shooting in ensembles: 002 with paths: 79 and worker: 1
------- pipipi 91 END -------

------- pipipi 92 START -------
shooted in ensembles: 003 with paths: 77 -> 77 with status: NCR and worker: 0
===
ens	p_num	state			files
000	80 	[1. 0. 0. 0. 0. 0.]	['000_149_trajB.trr', 'second.g96']
001	81 	[0. 1. 1. 0. 0. 0.]	['001_150_trajF.trr', 'second_last.g96']
002	79 	[0. 1. 1. 0. 0. 0.]	['002_141_trajB.trr', '002_142_trajF.trr']
003	77 	[0. 1. 1. 1. 0. 0.]	['003_139_trajB.trr', '003_140_trajF.trr']
00

In [13]:
print('--------------------------------------------------')
print('live trajs:', live_trajs, f'after {stopping} cycles')
print('==================================================')
print('xx | 000        001     002     003     004      |')
print('--------------------------------------------------')
for key, item in traj_num_dic.items():
    print(f'{key:02.0f}', "|" if key not in live_trajs else '*',
          '\t'.join([f'{item0:02.2f}' if item0 != 0.0 else '---' for item0 in item['weight'][:-1]])
         ,'\t', "|" if key not in live_trajs else '*')

--------------------------------------------------
live trajs: [87, 86, 82, 77, 83] after 100 cycles
xx | 000        001     002     003     004      |
--------------------------------------------------
00 | ---	2.00	---	---	--- 	 |
01 | ---	---	0.50	0.50	--- 	 |
02 | ---	---	---	---	--- 	 |
03 | ---	---	---	---	--- 	 |
04 | 4.00	---	---	---	--- 	 |
05 | ---	0.50	9.00	10.50	--- 	 |
06 | ---	---	---	---	15.00 	 |
07 | ---	1.00	---	---	--- 	 |
08 | ---	0.50	8.00	8.50	--- 	 |
09 | 1.00	---	---	---	--- 	 |
10 | ---	1.00	---	---	--- 	 |
11 | 1.00	---	---	---	--- 	 |
12 | ---	3.00	---	---	--- 	 |
13 | 2.00	---	---	---	--- 	 |
14 | ---	1.00	---	---	--- 	 |
15 | 2.00	---	---	---	--- 	 |
16 | ---	0.50	0.50	---	--- 	 |
17 | 2.00	---	---	---	--- 	 |
18 | ---	1.00	---	---	--- 	 |
19 | 3.00	---	---	---	--- 	 |
20 | ---	1.50	0.50	---	--- 	 |
21 | ---	---	---	---	12.00 	 |
22 | ---	1.50	2.50	15.00	--- 	 |
23 | 4.00	---	---	---	--- 	 |
24 | ---	4.00	---	---	--- 	 |
25 | ---	2.00	3.50	3.50	--- 	 |
26 |